In [1]:
from filter_functions import image_filter

Dashboard at http://127.0.0.1:8787/status


In [2]:
from training_functions import train_segmentation

In [25]:
import dask
import xarray as xr
import numpy as np
import  matplotlib.pyplot as plt
import os
from ipywidgets import Image
from ipywidgets import ColorPicker, IntSlider, link, AppLayout, HBox
from ipycanvas import  hold_canvas,  MultiCanvas #RoughCanvas,Canvas,

In [4]:
path = '/home/fische_r/NAS/testing/Jeremy_tomo/tomodata.nc'
path = r"C:\Zwischenlager\tomodata.nc"

In [5]:
IF = image_filter(data_path=path)

In [6]:
IF.open_raw_data()

In [7]:
IF.data

dask.array<array, shape=(200, 39, 233, 8), dtype=float64, chunksize=(50, 39, 91, 8), chunktype=numpy.ndarray>

In [8]:
IF.prepare()

In [9]:
IF.stack_features()

In [10]:
IF.feature_stack

,Array,Chunk
Bytes,7.91 GiB,10.83 MiB
Shape,"(200, 39, 233, 8, 73)","(50, 39, 91, 8, 1)"
Count,360 Graph Layers,876 Chunks
Type,float64,numpy.ndarray


In [11]:
IF.make_xarray_nc()

In [12]:
# IF.result

In [13]:
# IF.compute()

In [14]:
# IF.feature_stack.persist()

In [15]:
# test = IF.result['feature_stack'].sel(x=0,time=0).data

In [16]:
# test

In [17]:
training_path = r"C:\Zwischenlager\Jeremy_tomo"
if not os.path.exists(training_path):
    os.mkdir(training_path)

In [18]:
TS = train_segmentation(training_path=training_path)

In [19]:
TS.import_lazy_feature_stack(IF.result)

In [20]:
TS.suggest_training_set()

You could try  y = 27  and  z = 231


### iterative training

In [21]:
c1 = 'y'
p1 = 10
c2 = 'time'
p2 = 0

In [22]:
TS.load_training_set(c1, p1, c2, p2)

C:\git_repos\01_Python\pytrainseg\training_functions.py:173: RuntimeWarning: invalid value encountered in divide
  diff = diff/diff.max()*255


In [23]:
TS.interface()

In [26]:
# TS.current_im8 = TS.current_im8.compute()